In [1]:
!pip install gurobipy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 4.7 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
import os
import sys
# sys.path.append('../')
sys.path.append('/content/gdrive/MyDrive/SVR_Codes/SVR_CG')
from Codes.model import *
from Codes.utils import *

plt.style.use('default')

from tqdm import tqdm
import pickle
sys.path.append('/content/gdrive/MyDrive/SVR_Codes/ExptsEval')
RESULT_DIR = "/content/gdrive/MyDrive/SVR_Codes/ExptsEval"

from timeit import default_timer
from sklearn.datasets import make_blobs




In [4]:
centers = np.array([[1.92808404],
       [0.69454242] ,
       [10.63518777]])

In [5]:
def Blobs(N,D,K,box,seed,ratio, centers, ratio_std_x = 0.01, ratio_std_y=0.0):

    N1 = int(N/(ratio+1))
    N2 = N1*ratio
    split = K//2+1
    print(split)
    generator = np.random.RandomState(seed)
    regParam = np.round(generator.uniform(-3,3, size=(K, D+1)),2)
    # print(regParam)
    # box = 1
    generator = np.random.RandomState(seed)
    # centers = generator.uniform(0, box, size=(K, D))
    # print(centers)
    # ratio_std_x = 0.01
    # ratio_std_y = 0.0

    # X, y = make_blobs(n_samples=N, centers=centers, n_features=D, cluster_std = ratio_std_x,
    #                     random_state=seed)
    
    X, y= make_blobs(n_samples=N1, centers=centers[0:split], n_features=D, cluster_std = ratio_std_x,random_state=seed)

    # Dense Blobs
    X1, y1 = make_blobs(n_samples=N2, centers=centers[split:], n_features=D, cluster_std = ratio_std_x,
                    random_state=seed)

    y1 = y1+(split)    

    X = np.concatenate((X,X1), axis = 0)
    y = np.concatenate((y,y1), axis = 0)

    # print(centers)
    cols = [ 'X' + str(i+1) for i in range(D) ]
    df_data = pd.DataFrame(X, columns = cols)

    for k in range(K):
        generator = np.random.RandomState(seed)
        df_data.loc[y==k,'Y'] = X[y == k] @ regParam[k,:-1] + regParam[k,-1] + generator.normal(0,ratio_std_y,X[y==k].shape[0])

    df_data.plot('X1', 'Y', kind = 'scatter', s = 20)
    plt.show()

    return df_data, X,y


## For milp rg

In [ ]:
N=10000
K=3
D = 1
box = 100
seed =78
ratio_std_x=0.5
ratio_std_y=0.001

ratio_list = [1, 10, 20, 30, 40, 50 ]

cnt = 0


optgap_list=[]

time_list = []
metrics_list = []

# obj_list = []
constrsCnt_list = []

optVal_list = []

weights_list = []
bias_list = []

for i, ratio in enumerate(tqdm(ratio_list)):

    
      print('\nStarting Dense/Sparse ratio = %i' %(ratio))

      df_data, X,y = Blobs(N,D,K,box,seed,ratio, centers,ratio_std_x, ratio_std_y)


      clus = CLR(K =K , f = D, max_iter= 20, compute=True, random_state=234)
      clus.set_model(Cl_SVR_Cg(outputFlag=False, initConstrCnt=7, optimalGap= 0.05, tol = 0.002, time = 10, initConstrIntRatio= 10, step_plots=False, outliersCnt = 0))


      clus.fit(df_data )

      metrics_list.append(clus.evaluate(y))
      # print(metrics_list[-1])
      time_list.append(clus.run_time)

      constrsCnt_list.append(len(clus.constr_gen_pts))
      optgap_list.append(clus.model_optGap)

      optVal_list.append(clus.model_optVal)

      weights_list.append(clus.weights)
      bias_list.append(clus.bias)


      # obj_list.append(clus)

      cnt+=1
      print('MaxError: ', optVal_list[-1])
      print('Finished Dense/Sparse ratio = %i' %(ratio))
      print('*'*100)


with open(os.path.join(RESULT_DIR, 'SynImb2_'+str(K)+str(D)+'_clsvr.pkl'), 'wb') as outp:

    pickle.dump(metrics_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(time_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(constrsCnt_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(optgap_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(optVal_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(weights_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(bias_list, outp, pickle.HIGHEST_PROTOCOL)





## For EM greedy

In [ ]:
N=10000
K=3
D = 1
box = 100
seed =78
ratio_std_x=0.5
ratio_std_y=0.001

ratio_list = [1, 10, 20, 30, 40, 50 ]

cnt = 0

time_list = []
metrics_list = []



optVal_list = []

weights_list = []
bias_list = []

for i, ratio in enumerate(tqdm(ratio_list)):

    
      print('\nStarting Dense/Sparse ratio = %i' %(ratio))

      time_list_inner = []
      metrics_list_inner = []

      optVal_list_inner = []

      weights_list_inner = []
      bias_list_inner = []

      df_data, X,y = Blobs(N,D,K,box,seed,ratio, centers,ratio_std_x, ratio_std_y)

      for j in range(10):

        greedy = CLR(K =K , f = D, max_iter= 30, compute = True, random_state=seed )
        greedy.set_model(Cl_SVR_greedy(Kmeans_init = False, outputFlag=False, tol = 0.05, time = 0.5, step_plots=False))

        greedy.fit(df_data )

        metrics_list_inner.append(greedy.evaluate(y))
        # print(metrics_list[-1])
        time_list_inner.append(greedy.run_time)


        optVal_list_inner.append(greedy.model_optVal)

        weights_list_inner.append(greedy.weights)
        bias_list_inner.append(greedy.bias)


      time_list.append(np.average(time_list_inner))
      metrics_list.append(np.average(metrics_list_inner, axis = 0))
      optVal_list.append(np.average(optVal_list_inner))

      weights_list.append(weights_list_inner)
      bias_list.append(bias_list_inner)


      # obj_list.append(clus)

      cnt+=1
      print('MaxError: ', optVal_list[-1])
      print('Finished Dense/Sparse ratio = %i' %(ratio))
      print('*'*100)


with open(os.path.join(RESULT_DIR, 'SynImb2_'+str(K)+str(D)+'_greedy.pkl'), 'wb') as outp:

    pickle.dump(metrics_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(time_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(optVal_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(weights_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(bias_list, outp, pickle.HIGHEST_PROTOCOL)





## K plane 

In [ ]:
N=10000
K=3
D = 1
box = 100
seed =78
ratio_std_x=0.5
ratio_std_y=0.001

ratio_list = [1, 10, 20, 30, 40, 50 ]

cnt = 0

time_list = []
metrics_list = []



optVal_list = []

weights_list = []
bias_list = []

for i, ratio in enumerate(tqdm(ratio_list)):

    
      print('\nStarting Dense/Sparse ratio = %i' %(ratio))

      time_list_inner = []
      metrics_list_inner = []

      optVal_list_inner = []

      weights_list_inner = []
      bias_list_inner = []

      df_data, X,y = Blobs(N,D,K,box,seed,ratio, centers,ratio_std_x, ratio_std_y)

      for j in range(10):

        kp = CLR(K =K , f = D, max_iter= 20, random_state=seed )
        kp.set_model(k_plane())
        kp.fit(df_data )

        metrics_list_inner.append(kp.evaluate(y))
        # print(metrics_list[-1])
        time_list_inner.append(kp.run_time)


        optVal_list_inner.append(kp.model_optVal)

        weights_list_inner.append(kp.weights)
        bias_list_inner.append(kp.bias)


      time_list.append(np.average(time_list_inner))
      metrics_list.append(np.average(metrics_list_inner, axis = 0))
      optVal_list.append(np.average(optVal_list_inner))

      weights_list.append(weights_list_inner)
      bias_list.append(bias_list_inner)


      # obj_list.append(clus)

      cnt+=1
      print('MaxError: ', optVal_list[-1])
      print('Finished Dense/Sparse ratio = %i' %(ratio))
      print('*'*100)


with open(os.path.join(RESULT_DIR, 'SynImb2_'+str(K)+str(D)+'kp.pkl'), 'wb') as outp:

    pickle.dump(metrics_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(time_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(optVal_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(weights_list, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(bias_list, outp, pickle.HIGHEST_PROTOCOL)



